# TimeGPT

In [ ]:
#| default_exp timegpt

In [ ]:
#| hide 
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import logging
import inspect
import json
import requests
import warnings
from typing import Dict, List, Optional, Union

import numpy as np
import pandas as pd

from nixtlats.client import Nixtla, SingleSeriesForecast

logging.basicConfig(level=logging.INFO)
main_logger = logging.getLogger(__name__)
httpx_logger = logging.getLogger('httpx')
httpx_logger.setLevel(logging.ERROR)

In [ ]:
#| hide
import os
import warnings
from itertools import product

from dotenv import load_dotenv
from fastcore.test import test_eq, test_fail, test_warns
from nbdev.showdoc import show_doc
from tqdm import TqdmExperimentalWarning

load_dotenv()
logging.getLogger('statsforecast').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)

In [ ]:
#| exporti
date_features_by_freq = {
    # Daily frequencies
    'B': ['year', 'month', 'day', 'weekday'],
    'C': ['year', 'month', 'day', 'weekday'],
    'D': ['year', 'month', 'day', 'weekday'],
    # Weekly
    'W': ['year', 'week', 'weekday'],
    # Monthly
    'M': ['year', 'month'],
    'SM': ['year', 'month', 'day'],
    'BM': ['year', 'month'],
    'CBM': ['year', 'month'],
    'MS': ['year', 'month'],
    'SMS': ['year', 'month', 'day'],
    'BMS': ['year', 'month'],
    'CBMS': ['year', 'month'],
    # Quarterly
    'Q': ['year', 'quarter'],
    'BQ': ['year', 'quarter'],
    'QS': ['year', 'quarter'],
    'BQS': ['year', 'quarter'],
    # Yearly
    'A': ['year'],
    'Y': ['year'],
    'BA': ['year'],
    'BY': ['year'],
    'AS': ['year'],
    'YS': ['year'],
    'BAS': ['year'],
    'BYS': ['year'],
    # Hourly
    'BH': ['year', 'month', 'day', 'hour', 'weekday'],
    'H': ['year', 'month', 'day', 'hour'],
    # Minutely
    'T': ['year', 'month', 'day', 'hour', 'minute'],
    'min': ['year', 'month', 'day', 'hour', 'minute'],
    # Secondly
    'S': ['year', 'month', 'day', 'hour', 'minute', 'second'],
    # Milliseconds
    'L': ['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond'],
    'ms': ['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond'],
    # Microseconds
    'U': ['year', 'month', 'day', 'hour', 'minute', 'second', 'microsecond'],
    'us': ['year', 'month', 'day', 'hour', 'minute', 'second', 'microsecond'],
    # Nanoseconds
    'N': []
}

In [ ]:
#| exporti
class _TimeGPTModel:

    def __init__(
            self, 
            client: Nixtla,
            h: int,
            id_col: str = 'unique_id',
            time_col: str = 'ds',
            target_col: str = 'y',
            freq: str = None,
            level: Optional[List[Union[int, float]]] = None,
            finetune_steps: int = 0,
            clean_ex_first: bool = True,
            date_features: Union[bool, List[str]] = False,
            date_features_to_one_hot: Union[bool, List[str]] = True,
        ):
        self.client = client
        self.h = h
        self.id_col = id_col
        self.time_col = time_col
        self.target_col = target_col
        self.base_freq = freq
        self.level = level
        self.finetune_steps = finetune_steps
        self.clean_ex_first = clean_ex_first
        self.date_features = date_features
        self.date_features_to_one_hot = date_features_to_one_hot
        # variables defined by each flow
        self.weights_x: pd.DataFrame = None
        self.freq: str = self.base_freq
        self.drop_uid: bool = False
        self.x_cols: List[str]
        self.input_size: int
        self.model_horizon: int


    def transform_inputs(self, df: pd.DataFrame, X_df: pd.DataFrame):
        df = df.copy()
        main_logger.info('Validating inputs...')
        if self.base_freq is None and hasattr(df.index, 'freq'):
            inferred_freq = df.index.freq
            if inferred_freq is not None:
                inferred_freq = inferred_freq.rule_code
                main_logger.info(f'Inferred freq: {inferred_freq}')
            self.freq = inferred_freq
            time_col = df.index.name
            if time_col is None:
                time_col = 'ds'
                df.index.name = time_col
            self.time_col = time_col
            df = df.reset_index()
        else:
            self.freq = self.base_freq
        renamer = {
            self.id_col: 'unique_id',
            self.time_col: 'ds',
            self.target_col: 'y',
        }
        df = df.rename(columns=renamer)
        if df.dtypes.ds != 'object':
            df['ds'] = df['ds'].astype(str)
        if 'unique_id' not in df.columns:
            # Insert unique_id column
            df = df.assign(unique_id='ts_0')
            self.drop_uid = True
        if X_df is not None:
            X_df = X_df.rename(columns=renamer)
            if 'unique_id' not in X_df.columns:
                X_df = X_df.assign(unique_id='ts_0')
            if X_df.dtypes.ds != 'object':
                X_df['ds'] = X_df['ds'].astype(str)
        return df, X_df

    def transform_outputs(self, fcst_df: pd.DataFrame):
        renamer = {
            'unique_id': self.id_col,
            'ds': self.time_col,
            'target_col': self.target_col,
        }
        if self.drop_uid:
            fcst_df = fcst_df.drop(columns='unique_id')
        fcst_df = fcst_df.rename(columns=renamer)
        return fcst_df

    def infer_freq(self, df: pd.DataFrame):
        # special freqs that need to be checked
        # for example to ensure 'W'-> 'W-MON'
        special_freqs = ['W', 'M', 'Q', 'Y', 'A']
        if self.freq is None or self.freq in special_freqs:
            unique_id = df.iloc[0]['unique_id']
            df_id = df.query('unique_id == @unique_id')
            inferred_freq = pd.infer_freq(df_id['ds'])
            if inferred_freq is None:
                raise Exception(
                    'Could not infer frequency of ds column. This could be due to '
                    'inconsistent intervals. Please check your data for missing, '
                    'duplicated or irregular timestamps'
                )
            if self.freq is not None:
                # check we have the same base frequency
                # except when we have yearly frequency (A, and Y means the same)
                if (self.freq != inferred_freq[0] and self.freq != 'Y') or (self.freq == 'Y' and inferred_freq[0] != 'A'):
                    raise Exception(f'Failed to infer special date, inferred freq {inferred_freq}')
            main_logger.info(f'Inferred freq: {inferred_freq}')
            self.freq = inferred_freq

    def resample_dataframe(self, df: pd.DataFrame):
        df = df.copy()
        df['ds'] = pd.to_datetime(df['ds'])
        resampled_df = df.set_index('ds').groupby('unique_id').resample(self.freq).bfill()
        resampled_df = resampled_df.drop(columns='unique_id').reset_index()
        resampled_df['ds'] = resampled_df['ds'].astype(str)
        return resampled_df

    def make_future_dataframe(self, df: pd.DataFrame, reconvert: bool = True):
        last_dates = df.groupby('unique_id')['ds'].max()
        def _future_date_range(last_date):
            future_dates = pd.date_range(last_date, freq=self.freq, periods=self.h + 1)
            future_dates = future_dates[-self.h:]
            return future_dates
        future_df = last_dates.apply(_future_date_range).reset_index()
        future_df = future_df.explode('ds').reset_index(drop=True)
        if reconvert and df.dtypes['ds'] == 'object':
            # avoid date 000
            future_df['ds'] = future_df['ds'].astype(str)
        return future_df

    def compute_date_feature(self, dates, feature):
        if callable(feature):
            feat_name = feature.__name__
            feat_vals = feature(dates)
        else:
            feat_name = feature
            if feature in ("week", "weekofyear"):
                dates = dates.isocalendar()
            feat_vals = getattr(dates, feature)
        if not isinstance(feat_vals, pd.DataFrame):
            vals = np.asarray(feat_vals)
            feat_vals = pd.DataFrame({feat_name: vals})
        feat_vals['ds'] = dates
        return feat_vals

    def add_date_features( 
            self,
            df: pd.DataFrame,
            X_df: Optional[pd.DataFrame],
        ):
        # df contains exogenous variables
        # X_df are the future values of the exogenous variables
        # construct dates
        train_dates = df['ds'].unique().tolist()
        # if we dont have future exogenos variables
        # we need to compute the future dates
        if (self.h is not None) and X_df is None:
            X_df = self.make_future_dataframe(df=df)
            future_dates = X_df['ds'].unique().tolist()
        elif X_df is not None:
            future_dates = X_df['ds'].unique().tolist()
        else:
            future_dates = []
        dates = pd.DatetimeIndex(train_dates + future_dates)
        date_features_df = pd.DataFrame({'ds': dates})
        for feature in self.date_features:
            feat_df = self.compute_date_feature(dates, feature)
            date_features_df = date_features_df.merge(feat_df, on=['ds'], how='left')
        if df.dtypes['ds'] == 'object':
            date_features_df['ds'] = date_features_df['ds'].astype(str)
        if self.date_features_to_one_hot is not None:
            date_features_df = pd.get_dummies(
                date_features_df, 
                columns=self.date_features_to_one_hot, 
                dtype=int,
            )
        # remove duplicated columns if any
        date_features_df = date_features_df.drop(
            columns=[col for col in date_features_df.columns if col in df.columns and col not in ['unique_id', 'ds']]
        )
        # add date features to df
        df = df.merge(date_features_df, on='ds', how='left')
        # add date features to X_df
        if X_df is not None:
            X_df = X_df.merge(date_features_df, on='ds', how='left')
        return df, X_df

    def preprocess_X_df(self, X_df: pd.DataFrame):
        if X_df.isna().any().any():
            raise Exception('Some of your exogenous variables contain NA, please check')
        X_df = X_df.sort_values(['unique_id', 'ds']).reset_index(drop=True)
        X_df = self.resample_dataframe(X_df)
        return X_df

    def preprocess_dataframes(
            self, 
            df: pd.DataFrame, 
            X_df: Optional[pd.DataFrame],
        ):
        self.infer_freq(df=df)
        """Returns Y_df and X_df dataframes in the structure expected by the endpoints."""

        y_cols = ['unique_id', 'ds', 'y']
        Y_df = df[y_cols]
        if Y_df['y'].isna().any():
            raise Exception('Your target variable contains NA, please check')
        # Azul: efficient this code
        # and think about returning dates that are not in the training set
        Y_df = self.resample_dataframe(Y_df)
        x_cols = []
        if X_df is not None:
            x_cols = X_df.drop(columns=['unique_id', 'ds']).columns.to_list()
            if not all(col in df.columns for col in x_cols):
                raise Exception(
                    'You must include the exogenous variables in the `df` object, '
                    f'exogenous variables {",".join(x_cols)}'
                )
            if (self.h is not None) and (len(X_df) != df['unique_id'].nunique() * self.h):
                raise Exception(
                    f'You have to pass the {self.h} future values of your '
                    'exogenous variables for each time series'
                )
            X_df_history = df[['unique_id', 'ds'] + x_cols]
            X_df = pd.concat([X_df_history, X_df])
            X_df = self.preprocess_X_df(X_df)
        elif (X_df is None) and (self.h is None) and (len(y_cols) < df.shape[1]):
            # case for just insample, 
            # we dont need h
            X_df = df.drop(columns='y')
            x_cols = X_df.drop(columns=['unique_id', 'ds']).columns.to_list()
            X_df = self.preprocess_X_df(X_df)
        self.x_cols = x_cols
        return Y_df, X_df

    def dataframes_to_dict(self, Y_df: pd.DataFrame, X_df: pd.DataFrame):
        to_dict_args = {'orient': 'split'}
        if 'index' in inspect.signature(pd.DataFrame.to_dict).parameters:
            to_dict_args['index'] = False
        y = Y_df.to_dict(**to_dict_args)
        x = X_df.to_dict(**to_dict_args) if X_df is not None else None
        return y, x
    
    def set_model_params(self):
        model_params = self.client.timegpt_model_params(request=SingleSeriesForecast(freq=self.freq))
        if 'data' in model_params:
            model_params = model_params['data']
        model_params = model_params['detail']
        self.input_size, self.model_horizon = model_params['input_size'], model_params['horizon']
        if self.h > self.model_horizon:
            main_logger.warning(
                'The specified horizon "h" exceeds the model horizon. '
                'This may lead to less accurate forecasts. '
                'Please consider using a smaller horizon.'
            )
    
    def validate_input_size(self, Y_df: pd.DataFrame):
        min_history = Y_df.groupby('unique_id').size().min()
        if min_history < self.input_size + self.model_horizon:
            raise Exception(
                'Your time series data is too short '
                'Please be sure that your unique time series contain '
                f'at least {self.input_size + self.model_horizon} observations'
            )

    def forecast(
        self,
        df: pd.DataFrame,
        X_df: Optional[pd.DataFrame] = None,
        add_history: bool = False,
    ):
        df, X_df = self.transform_inputs(df=df, X_df=X_df)
        main_logger.info('Preprocessing dataframes...')
        Y_df, X_df = self.preprocess_dataframes(df=df, X_df=X_df)
        self.set_model_params()
        # restrict input if
        # - we dont want to finetune
        # - we dont have exogenous regegressors
        # - and we dont want to produce pred intervals
        # - no add history
        restrict_input = self.finetune_steps == 0 and X_df is None and self.level is not None and not add_history
        if restrict_input:
            # add sufficient info to compute
            # conformal interval
            new_input_size = 3 * self.input_size + max(self.model_horizon, self.h)
            Y_df = Y_df.groupby('unique_id').tail(new_input_size)
            if X_df is not None:
                X_df = X_df.groupby('unique_id').tail(new_input_size + self.h) # history plus exogenous
        if self.finetune_steps > 0 or self.level is not None:
            self.validate_input_size(Y_df=Y_df)
        y, x = self.dataframes_to_dict(Y_df, X_df)
        print(y, x)
        print(self.x_cols)
        main_logger.info('Calling Forecast Endpoint...')
        response_timegpt = self.client.timegpt_multi_series(
            y=y,
            x=x,
            fh=self.h,
            freq=self.freq,
            level=self.level,
            finetune_steps=self.finetune_steps,
            clean_ex_first=self.clean_ex_first,
        )
        if 'data' in response_timegpt:
            response_timegpt = response_timegpt['data']
        if 'weights_x' in response_timegpt:
            print(response_timegpt['weights_x'])
            self.weights_x = pd.DataFrame({
                'features': self.x_cols,
                'weights': response_timegpt['weights_x'],
            })
        fcst_df = pd.DataFrame(**response_timegpt['forecast'])
        if add_history:
            main_logger.info('Calling Historical Forecast Endpoint...')
            self.validate_input_size(Y_df=Y_df)
            if 'data' in response_timegpt:
                response_timegpt = response_timegpt['data']
            response_timegpt = self.client.timegpt_multi_series_historic(
                y=y,
                x=x,
                freq=self.freq,
                level=self.level,
                clean_ex_first=self.clean_ex_first,
            )
            fitted_df = pd.DataFrame(**response_timegpt['forecast'])
            fitted_df = fitted_df.drop(columns='y')
            fcst_df = pd.concat([fitted_df, fcst_df]).sort_values(['unique_id', 'ds'])
        fcst_df = self.transform_outputs(fcst_df)
        return fcst_df

    def detect_anomalies(self, df: pd.DataFrame):
        # Azul
        # Remember the input X_df is the FUTURE ex vars
        # there is a misleading notation here
        # because X_df inputs in the following methods
        # returns X_df outputs that means something different
        # ie X_df = [X_df_history, X_df]
        # exogenous variables are passed after df 
        df, _ = self.transform_inputs(df=df, X_df=None)
        main_logger.info('Preprocessing dataframes...')
        Y_df, X_df = self.preprocess_dataframes(df=df, X_df=None)
        main_logger.info('Calling Anomaly Detector Endpoint...')
        y, x = self.dataframes_to_dict(Y_df, X_df)
        response_timegpt = self.client.timegpt_multi_series_anomalies(
            y=y,
            x=x,
            freq=self.freq,
            level=[self.level] if (isinstance(self.level, int) or isinstance(self.level, float)) else [self.level[0]],
            clean_ex_first=self.clean_ex_first,
        )
        if 'data' in response_timegpt:
            response_timegpt = response_timegpt['data']
        if 'weights_x' in response_timegpt:
            self.weights_x = pd.DataFrame({
                'features': self.x_cols,
                'weights': response_timegpt['weights_x'],
            })
        anomalies_df = pd.DataFrame(**response_timegpt['forecast'])
        anomalies_df = anomalies_df.drop(columns='y')
        anomalies_df = self.transform_outputs(anomalies_df)
        return anomalies_df

    def cross_validation(self, df: pd.DataFrame, n_windows: int, step_size: int):
        # Azul
        # Remember the input X_df is the FUTURE ex vars
        # there is a misleading notation here
        # because X_df inputs in the following methods
        # returns X_df outputs that means something different
        # ie X_df = [X_df_history, X_df]
        # exogenous variables are passed after df 
        df, _ = self.transform_inputs(df=df, X_df=None)
        main_logger.info('Preprocessing dataframes...')
        Y_df, X_df = self.preprocess_dataframes(df=df, X_df=None)
        main_logger.info('Calling Cross Validation Endpoint...')
        y, x = self.dataframes_to_dict(Y_df, X_df)
        print(y, x)
        print(self.x_cols)
        response_timegpt = self.client.timegpt_multi_series_cross_validation(
            y=y,
            x=x,
            fh=self.h,
            freq=self.freq,
            level=self.level,
            n_windows=n_windows,
            step_size=step_size,
            finetune_steps=self.finetune_steps,
            clean_ex_first=self.clean_ex_first,
        )
        if 'data' in response_timegpt:
            response_timegpt = response_timegpt['data']
        if 'weights_x' in response_timegpt:
            self.weights_x = pd.DataFrame({
                'features': self.x_cols,
                'weights': response_timegpt['weights_x'],
            })
        cv_df = pd.DataFrame(**response_timegpt['forecast'])
        cv_df = self.transform_outputs(cv_df)
        return cv_df

In [ ]:
#| exporti
class _TimeGPT:
    """
    A class used to interact with the TimeGPT API.
    """

    def __init__(self, token: str, environment: Optional[str] = None):
        """
        Constructs all the necessary attributes for the TimeGPT object.

        Parameters
        ----------
        token : str
            The authorization token to interact with the TimeGPT API.
        environment : str
            Custom environment. Pass only if provided.
        """
        if environment is None:
            environment = "https://dashboard.nixtla.io/api"
        self.client = Nixtla(base_url=environment, token=token)
        self.weights_x: pd.DataFrame = None

    def validate_token(self, log: bool = True) -> bool:
        """Returns True if your token is valid."""
        validation = self.client.validate_token()
        valid = False
        if 'message' in validation:
            if validation['message'] == 'success':
                valid = True
        elif 'detail' in validation:
            if 'Forecasting! :)' in validation['detail']:
                valid = True
        if 'support' in validation and log:
            main_logger.info(f'Happy Forecasting! :), {validation["support"]}')
        return valid
        
    def _forecast(
            self,
            df: pd.DataFrame,
            h: int,
            freq: Optional[str] = None,    
            id_col: str = 'unique_id',
            time_col: str = 'ds',
            target_col: str = 'y',
            X_df: Optional[pd.DataFrame] = None,
            level: Optional[List[Union[int, float]]] = None,
            finetune_steps: int = 0,
            clean_ex_first: bool = True,
            validate_token: bool = False,
            add_history: bool = False,
            date_features: Union[bool, List[str]] = False,
            date_features_to_one_hot: Union[bool, List[str]] = True,
        ):
        """Forecast your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we 
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        h : int
            Forecast horizon.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        X_df : pandas.DataFrame, optional (default=None)
            DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous.
        level : List[float], optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        finetune_steps : int (default=0)
            Number of steps used to finetune TimeGPT in the
            new data.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        validate_token : bool (default=False)
            If True, validates token before 
            sending requests.
        add_history : bool (default=False)
            Return fitted values of the model.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        date_features_to_one_hot : bool or list of str (default=True)
            Apply one-hot encoding to these date features.
            If `date_features=True`, then all date features are
            one-hot encoded by default.
        
        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with TimeGPT forecasts for point predictions and probabilistic
            predictions (if level is not None).
        """
        if validate_token and not self.validate_token(log=False):
            raise Exception(
                'Token not valid, please email ops@nixtla.io'
            )
        self.weights_x = None
        model = _TimeGPTModel(
            client=self.client,
            h=h,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            freq=freq,
            level=level,
            finetune_steps=finetune_steps,
            clean_ex_first=clean_ex_first,
            date_features=date_features,
            date_features_to_one_hot=date_features_to_one_hot,
        )
        fcst_df = model.forecast(df=df, X_df=X_df, add_history=add_history)
        self.weights_x = model.weights_x
        return fcst_df

    def _detect_anomalies(
            self,
            df: pd.DataFrame,
            freq: Optional[str] = None,    
            id_col: str = 'unique_id',
            time_col: str = 'ds',
            target_col: str = 'y',
            level: Union[int, float] = 99,
            clean_ex_first: bool = True,
            validate_token: bool = False,
            date_features: Union[bool, List[str]] = False,
            date_features_to_one_hot: Union[bool, List[str]] = True,
        ):
        """Detect anomalies in your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we 
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        level : float (default=99)
            Confidence level between 0 and 100 for detecting the anomalies.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        validate_token : bool (default=False)
            If True, validates token before 
            sending requests.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        date_features_to_one_hot : bool or list of str (default=True)
            Apply one-hot encoding to these date features.
            If `date_features=True`, then all date features are
            one-hot encoded by default.
        
        Returns
        -------
        anomalies_df : pandas.DataFrame
            DataFrame with anomalies flagged with 1 detected by TimeGPT.
        """
        if validate_token and not self.validate_token(log=False):
            raise Exception(
                'Token not valid, please email ops@nixtla.io'
            )
        self.weights_x = None
        model = _TimeGPTModel(
            client=self.client,
            h=None,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            freq=freq,
            level=level,
            clean_ex_first=clean_ex_first,
            date_features=date_features,
            date_features_to_one_hot=date_features_to_one_hot,
        )
        anomalies_df = model.detect_anomalies(df=df)
        self.weights_x = model.weights_x
        return anomalies_df

    def _cross_validation(
        self,
        df: pd.DataFrame,
        h: int,
        freq: Optional[str] = None,
        id_col: str = "unique_id",
        time_col: str = "ds",
        target_col: str = "y",
        level: Optional[List[Union[int, float]]] = None,
        validate_token: bool = False,
        n_windows: int = 1,
        step_size: Optional[int] = None,
        finetune_steps: int = 0,
        clean_ex_first: bool = True,
        date_features: Union[bool, List[str]] = False,
        date_features_to_one_hot: Union[bool, List[str]] = True,
    ):
        """Perform cross validation in your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        h : int
            Forecast horizon.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        level : float (default=99)
            Confidence level between 0 and 100 for detecting the anomalies.
        validate_token : bool (default=False)
            If True, validates token before
            sending requests.
        n_windows : int (defaul=1)
            Number of windows to evaluate.
        step_size : int, optional (default=None)
            Step size between each cross validation window. If None it will be equal to `h`.
        finetune_steps : int (default=0)
            Number of steps used to finetune TimeGPT in the
            new data.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        date_features_to_one_hot : bool or list of str (default=True)
            Apply one-hot encoding to these date features.
            If `date_features=True`, then all date features are
            one-hot encoded by default.

        Returns
        -------
        cv_df : pandas.DataFrame
            DataFrame with anomalies flagged with 1 detected by TimeGPT.
        """
        if validate_token and not self.validate_token(log=False):
            raise Exception(
                'Token not valid, please email ops@nixtla.io'
            )
        self.weights_x = None
        model = _TimeGPTModel(
            client=self.client,
            h=h,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            freq=freq,
            level=level,
            clean_ex_first=clean_ex_first,
            date_features=date_features,
            date_features_to_one_hot=date_features_to_one_hot,
        )
        cv_df = model.cross_validation(df=df, n_windows=n_windows, step_size=step_size)
        self.weights_x = model.weights_x
        return cv_df

    def plot(
            self,
            df: pd.DataFrame,
            forecasts_df: Optional[pd.DataFrame] = None,
            id_col: str = 'unique_id',
            time_col: str = 'ds',
            target_col: str = 'y',
            unique_ids: Union[Optional[List[str]], np.ndarray] = None,
            plot_random: bool = True,
            models: Optional[List[str]] = None,
            level: Optional[List[float]] = None,
            max_insample_length: Optional[int] = None,
            plot_anomalies: bool = False,
            engine: str = 'matplotlib',
            resampler_kwargs: Optional[Dict] = None,
        ):
        """Plot forecasts and insample values.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we 
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        forecasts_df : pandas.DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`, `ds`] and models.
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        unique_ids : List[str], optional (default=None)
            Time Series to plot.
            If None, time series are selected randomly.
        plot_random : bool (default=True)
            Select time series to plot randomly.
        models : List[str], optional (default=None)
            List of models to plot.
        level : List[float], optional (default=None)
            List of prediction intervals to plot if paseed.
        max_insample_length : int, optional (default=None)
            Max number of train/insample observations to be plotted.
        plot_anomalies : bool (default=False)
            Plot anomalies for each prediction interval.
        engine : str (default='plotly')
            Library used to plot. 'plotly', 'plotly-resampler' or 'matplotlib'.
        resampler_kwargs : dict
            Kwargs to be passed to plotly-resampler constructor.
            For further custumization ("show_dash") call the method,
            store the plotting object and add the extra arguments to
            its `show_dash` method.
        """
        try:
            from utilsforecast.plotting import plot_series
        except ModuleNotFoundError:
            raise Exception(
                'You have to install additional dependencies to use this method, '
                'please install them using `pip install "nixtlats[plotting]"`'
            )
        df = df.copy()
        if id_col not in df:
            df[id_col] = 'ts_0'
        df[time_col] = pd.to_datetime(df[time_col])
        if forecasts_df is not None:
            forecasts_df = forecasts_df.copy()
            if id_col not in forecasts_df:
                forecasts_df[id_col] = 'ts_0'
            forecasts_df[time_col] = pd.to_datetime(forecasts_df[time_col])
            if 'anomaly' in forecasts_df:
                # special case to plot outputs
                # from detect_anomalies
                forecasts_df = forecasts_df.drop(columns='anomaly')
                cols = forecasts_df.columns
                cols = cols[cols.str.contains('TimeGPT-lo-')]
                level = cols.str.replace('TimeGPT-lo-', '')[0]
                level = float(level) if '.' in level else int(level)
                level = [level]
                plot_anomalies = True
                models = ['TimeGPT']
                forecasts_df = df.merge(forecasts_df, how='left')
                df = df.groupby('unique_id').head(1)
                # prevent double plotting
                df.loc[:, target_col] = np.nan
        return plot_series(
            df=df,
            forecasts_df=forecasts_df,
            ids=unique_ids,
            plot_random=plot_random,
            models=models,
            level=level,
            max_insample_length=max_insample_length,
            plot_anomalies=plot_anomalies,
            engine=engine,
            resampler_kwargs=resampler_kwargs,
            palette="tab20b",
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
        )

In [ ]:
#| exporti
class TimeGPT(_TimeGPT):
    
    def forecast(
            self,
            df: pd.DataFrame,
            h: int,
            freq: Optional[str] = None,    
            id_col: str = 'unique_id',
            time_col: str = 'ds',
            target_col: str = 'y',
            X_df: Optional[pd.DataFrame] = None,
            level: Optional[List[Union[int, float]]] = None,
            finetune_steps: int = 0,
            clean_ex_first: bool = True,
            validate_token: bool = False,
            add_history: bool = False,
            date_features: Union[bool, List[str]] = False,
            date_features_to_one_hot: Union[bool, List[str]] = True,
            num_partitions: Optional[int] = None,
        ):
        """Forecast your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we 
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        h : int
            Forecast horizon.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        X_df : pandas.DataFrame, optional (default=None)
            DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous.
        level : List[float], optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        finetune_steps : int (default=0)
            Number of steps used to finetune TimeGPT in the
            new data.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        validate_token : bool (default=False)
            If True, validates token before 
            sending requests.
        add_history : bool (default=False)
            Return fitted values of the model.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        date_features_to_one_hot : bool or list of str (default=True)
            Apply one-hot encoding to these date features.
            If `date_features=True`, then all date features are
            one-hot encoded by default.
        num_partitions : int (default=None)
            Number of partitions to use.
            Only used in distributed environments (spark, ray, dask).
            If None, the number of partitions will be equal
            to the available parallel resources.
        
        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with TimeGPT forecasts for point predictions and probabilistic
            predictions (if level is not None).
        """
        if isinstance(df, pd.DataFrame):
            return self._forecast(
                df=df,
                h=h,
                freq=freq,    
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
                X_df=X_df,
                level=level,
                finetune_steps=finetune_steps,
                clean_ex_first=clean_ex_first,
                validate_token=validate_token,
                add_history=add_history,
                date_features=date_features,
                date_features_to_one_hot=date_features_to_one_hot,
            )
        else:
            from nixtlats.distributed.timegpt import _DistributedTimeGPT
            return _DistributedTimeGPT().forecast(
                token=self.client._client_wrapper._token,
                environment=self.client._client_wrapper._base_url,
                df=df,
                h=h,
                freq=freq,    
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
                X_df=X_df,
                level=level,
                finetune_steps=finetune_steps,
                clean_ex_first=clean_ex_first,
                validate_token=validate_token,
                add_history=add_history,
                date_features=date_features,
                date_features_to_one_hot=date_features_to_one_hot,
                num_partitions=num_partitions,
            )
            
    def detect_anomalies(
            self,
            df: pd.DataFrame,
            freq: Optional[str] = None,    
            id_col: str = 'unique_id',
            time_col: str = 'ds',
            target_col: str = 'y',
            level: Union[int, float] = 99,
            clean_ex_first: bool = True,
            validate_token: bool = False,
            date_features: Union[bool, List[str]] = False,
            date_features_to_one_hot: Union[bool, List[str]] = True,
        ):
        """Detect anomalies in your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we 
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        level : float (default=99)
            Confidence level between 0 and 100 for detecting the anomalies.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        validate_token : bool (default=False)
            If True, validates token before 
            sending requests.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        date_features_to_one_hot : bool or list of str (default=True)
            Apply one-hot encoding to these date features.
            If `date_features=True`, then all date features are
            one-hot encoded by default.
        
        Returns
        -------
        anomalies_df : pandas.DataFrame
            DataFrame with anomalies flagged with 1 detected by TimeGPT.
        """
        if isinstance(df, pd.DataFrame):
            return self._detect_anomalies(
                df=df,
                freq=freq,    
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
                level=level,
                clean_ex_first=clean_ex_first,
                validate_token=validate_token,
                date_features=date_features,
                date_features_to_one_hot=date_features_to_one_hot,
            )
        else:
            from nixtlats.distributed.timegpt import _DistributedTimeGPT
            return _DistributedTimeGPT().detect_anomalies(
                token=self.client._client_wrapper._token,
                environment=self.client._environment,
                df=df,
                freq=freq,    
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
                level=level,
                clean_ex_first=clean_ex_first,
                validate_token=validate_token,
                date_features=date_features,
                date_features_to_one_hot=date_features_to_one_hot,
                num_partitions=num_partitions,
            )

    def cross_validation(
        self,
        df: pd.DataFrame,
        h: int,
        freq: Optional[str] = None,
        id_col: str = "unique_id",
        time_col: str = "ds",
        target_col: str = "y",
        level: Optional[List[Union[int, float]]] = None,
        validate_token: bool = False,
        n_windows: int = 1,
        step_size: Optional[int] = None,
        finetune_steps: int = 0,
        clean_ex_first: bool = True,
        date_features: Union[bool, List[str]] = False,
        date_features_to_one_hot: Union[bool, List[str]] = True,
    ):
        """Perform cross validation in your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        h : int
            Forecast horizon.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        level : float (default=99)
            Confidence level between 0 and 100 for detecting the anomalies.
        validate_token : bool (default=False)
            If True, validates token before
            sending requests.
        n_windows : int (defaul=1)
            Number of windows to evaluate.
        step_size : int, optional (default=None)
            Step size between each cross validation window. If None it will be equal to `h`.
        finetune_steps : int (default=0)
            Number of steps used to finetune TimeGPT in the
            new data.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        date_features_to_one_hot : bool or list of str (default=True)
            Apply one-hot encoding to these date features.
            If `date_features=True`, then all date features are
            one-hot encoded by default.

        Returns
        -------
        cv_df : pandas.DataFrame
            DataFrame with cross validation fitted values.
        """
        if isinstance(df, pd.DataFrame):
            return self._cross_validation(
                df=df,
                h=h,
                freq=freq,
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
                level=level,
                validate_token=validate_token,
                n_windows=n_windows,
                step_size=step_size,
                finetune_steps=finetune_steps,
                clean_ex_first=clean_ex_first,
                date_features=date_features,
                date_features_to_one_hot=date_features_to_one_hot,
            )
        else:
            raise Exception(
                "Only pandas supported. "
                "Please raise an issue on Github or mail to ops@nixtla.io. "
                "Or ask Az."
            )

In [ ]:
show_doc(TimeGPT.__init__, title_level=3, name='TimeGPT')

---

### TimeGPT

>      TimeGPT (token:str, environment:Optional[str]=None)

Constructs all the necessary attributes for the TimeGPT object.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| token | str |  | The authorization token to interact with the TimeGPT API. |
| environment | Optional | None | Custom environment. Pass only if provided. |

In [ ]:
#| hide
timegpt = TimeGPT(token=os.environ['TIMEGPT_TOKEN'])

In [ ]:
show_doc(TimeGPT.validate_token, title_level=2, name='TimeGPT.validate_token')

---

## TimeGPT.validate_token

>      TimeGPT.validate_token (log:bool=True)

Returns True if your token is valid.

In [ ]:
#| hide
timegpt.validate_token()

INFO:__main__:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [ ]:
#| hide
_timegpt = TimeGPT(os.environ['TIMEGPT_CUSTOM_URL_TOKEN'], os.environ['TIMEGPT_CUSTOM_URL'])
_timegpt.validate_token()

True

In [ ]:
#| hide
test_fail(
    lambda: TimeGPT(token='transphobic').forecast(df=pd.DataFrame(), h=None, validate_token=True),
    contains='nixtla'
)

In [ ]:
#| hide
# test input_size
test_eq(
    timegpt.client.timegpt_model_params(request=SingleSeriesForecast(freq='D'))['data']['detail'],
    {'input_size': 28, 'horizon': 7},
)

Now you can start to make forecasts! Let's import an example:

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/air_passengers.csv')
df.head()

,timestamp,value
0,1949-01-01,112
1,1949-02-01,118
2,1949-03-01,132
3,1949-04-01,129
4,1949-05-01,121


In [ ]:
#| hide
# test pass dataframe with index
df_ds_index = df.set_index('timestamp')
df_ds_index.index = pd.DatetimeIndex(df_ds_index.index, freq='MS')
fcst_inferred_df_index = timegpt.forecast(df_ds_index, h=10, time_col='timestamp', target_col='value')
anom_inferred_df_index = timegpt.detect_anomalies(df_ds_index, time_col='timestamp', target_col='value')
fcst_inferred_df = timegpt.forecast(df, h=10, time_col='timestamp', target_col='value')
anom_inferred_df = timegpt.detect_anomalies(df, time_col='timestamp', target_col='value')
pd.testing.assert_frame_equal(fcst_inferred_df_index, fcst_inferred_df)
pd.testing.assert_frame_equal(anom_inferred_df_index, anom_inferred_df)
for freq in ['Y', 'W-MON', 'Q-DEC', 'H']:
    df_ds_index.index = pd.date_range(end='2023-01-01', periods=len(df), freq=freq)
    df_ds_index.index.name = 'timestamp'
    fcst_inferred_df_index = timegpt.forecast(df_ds_index, h=10, time_col='timestamp', target_col='value')
    df_test = df_ds_index.reset_index()
    fcst_inferred_df = timegpt.forecast(df_test, h=10, time_col='timestamp', target_col='value')
    pd.testing.assert_frame_equal(fcst_inferred_df_index, fcst_inferred_df)

INFO:__main__:Validating inputs...
INFO:__main__:Inferred freq: MS
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Calling Forecast Endpoint...
INFO:__main__:Validating inputs...
INFO:__main__:Inferred freq: MS
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Calling Anomaly Detector Endpoint...
INFO:__main__:Validating inputs...
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Inferred freq: MS
INFO:__main__:Calling Forecast Endpoint...
INFO:__main__:Validating inputs...
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Inferred freq: MS
INFO:__main__:Calling Anomaly Detector Endpoint...
INFO:__main__:Validating inputs...
INFO:__main__:Inferred freq: A-DEC
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Calling Forecast Endpoint...
INFO:__main__:Validating inputs...
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Inferred freq: A-DEC
INFO:__main__:Calling Forecast Endpoint...
INFO:__main__:Validating inputs...
INFO:__main__:Inferred freq: W-

In [ ]:
show_doc(TimeGPT.plot, name='TimeGPT.plot', title_level=2)

In [ ]:
#| hide
timegpt.plot(df, time_col='timestamp', target_col='value', engine='plotly')

In [ ]:
show_doc(TimeGPT.forecast, title_level=2)

In [ ]:
#| hide
# test infer freq
input_freqs = ['W', 'W', 'W', 'Q', 'Q', 'Q', 'M', 'M', 'Y', 'Y', 'Y']
expected_freqs = ['W-MON', 'W-TUE', 'W-WED', 'Q-DEC', 'QS-OCT', 'QS-DEC', 'MS', 'M', 'AS-JAN', 'A-DEC', 'AS-DEC']
for input_freq, output_freq in zip(input_freqs, expected_freqs):
    df_freq = pd.DataFrame({
        'unique_id': 'test_ts',
        'ds': pd.date_range('2021-01-01', periods=10, freq=output_freq),
    })
    model = _TimeGPTModel(client=timegpt.client, h=None, freq=input_freq)
    model.infer_freq(df_freq)
    test_eq(output_freq, model.freq)

In [ ]:
#| hide
# test make future dataframe for one series
df_ = df.rename(columns={'timestamp': 'ds', 'value': 'y'})
df_.insert(0, 'unique_id', 'AirPassengers')
df_actual_future = df_.tail(12)[['unique_id', 'ds']]
df_history = df_.drop(df_actual_future.index)
df_future = _TimeGPTModel(client=timegpt.client, h=12, freq='MS').make_future_dataframe(df_history)
pd.testing.assert_frame_equal(
    df_actual_future.reset_index(drop=True),
    df_future,
)

In [ ]:
#| hide
# test add date features
date_features = ['year', 'month']
df_date_features, future_df = _TimeGPTModel(
    client=timegpt.client,
    h=12, 
    freq='MS', 
    date_features=date_features,
    date_features_to_one_hot=None,
).add_date_features(df_,  X_df=None)
assert all(col in df_date_features for col in date_features)
assert all(col in future_df for col in date_features)

In [ ]:
#| hide
from nixtlats.date_features import SpecialDates

In [ ]:
#| hide
# test add callables
date_features = [SpecialDates({'first_dates': ['2021-01-1'], 'second_dates': ['2021-01-01']})]
df_daily = df_.copy()
df_daily['ds'] = pd.date_range(end='2021-01-01', periods=len(df_daily))
df_date_features, future_df = _TimeGPTModel(
    client=timegpt.client,
    h=12, 
    freq='D', 
    date_features=date_features,
    date_features_to_one_hot=None,
).add_date_features(df_,  X_df=None)
assert all(col in df_date_features for col in ['first_dates', 'second_dates'])
assert all(col in future_df for col in ['first_dates', 'second_dates'])

In [ ]:
#| hide
# test add date features one hot encoded
date_features = ['year', 'month']
date_features_to_one_hot = ['month']
df_date_features, future_df = _TimeGPTModel(
    client=timegpt.client,
    h=12, 
    freq='D', 
    date_features=date_features,
    date_features_to_one_hot=date_features_to_one_hot,
).add_date_features(df_,  X_df=None)

In [ ]:
#| hide
# test future dataframe for multiple series
df_ = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-with-ex-vars.csv')
df_actual_future = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-future-ex-vars.csv')
df_future = _TimeGPTModel(
        client=timegpt.client, 
        h=24, 
        freq='H',
    ).make_future_dataframe(df_[['unique_id', 'ds', 'y']])
pd.testing.assert_frame_equal(
    df_actual_future[['unique_id', 'ds']],
    df_future,
)

In [ ]:
show_doc(TimeGPT.cross_validation, title_level=2)

In [ ]:
#| hide
# cross validation tests
df_copy = df_.copy()
timegpt.cross_validation(df_, h=12, n_windows=2)
pd.testing.assert_frame_equal(
    df_copy,
    df_,
)
df_test = df_.groupby('unique_id').tail(12)
df_train = df_.drop(df_test.index)
hyps = [
    dict(),
    dict(date_features=['month']),
    dict(finetune_steps=2),
    dict(level=[80, 90]),
]
for hyp in hyps:
    print(hyp)
    fcst_test = timegpt.forecast(df_train, h=12, **hyp)
    fcst_test.insert(2, 'y', df_test['y'].values)
    fcst_cv = timegpt.cross_validation(df_, h=12, **hyp)
    pd.testing.assert_frame_equal(
        fcst_test,
        fcst_cv.drop(columns='cutoff'),
    )


INFO:__main__:Validating inputs...
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Inferred freq: MS
INFO:__main__:Calling Cross Validation Endpoint...


{'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143], 'columns': ['unique_id', 'ds', 'y'], 'data': [['AirPassengers', '1949-01-01', 112], ['AirPassengers', '1949-02-01', 118], ['AirPassengers', '1949-03-01', 132], ['AirPassengers', '1949-04-01', 129], ['AirPassengers', '1949-05-01', 121], ['AirPassengers', '1949-06-01', 135], ['AirPassengers', '1949-07-01', 148], ['AirPassengers', '1949-08-01', 148], ['AirPassengers', '1949-09-0

INFO:__main__:Validating inputs...
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Inferred freq: MS


{}


INFO:__main__:Calling Forecast Endpoint...


{'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131], 'columns': ['unique_id', 'ds', 'y'], 'data': [['AirPassengers', '1949-01-01', 112], ['AirPassengers', '1949-02-01', 118], ['AirPassengers', '1949-03-01', 132], ['AirPassengers', '1949-04-01', 129], ['AirPassengers', '1949-05-01', 121], ['AirPassengers', '1949-06-01', 135], ['AirPassengers', '1949-07-01', 148], ['AirPassengers', '1949-08-01', 148], ['AirPassengers', '1949-09-01', 136], ['AirPassengers', '1949-10-01', 119], ['AirPasseng

ValueError: All arrays must be of the same length

In [ ]:
timegpt.forecast(df_train, h=12, finetune_steps=2)

INFO:__main__:Validating inputs...
INFO:__main__:Preprocessing dataframes...
INFO:__main__:Inferred freq: MS


INFO:__main__:Calling Forecast Endpoint...


,unique_id,ds,TimeGPT
0,AirPassengers,1960-01-01,421.209320
1,AirPassengers,1960-02-01,409.877808
2,AirPassengers,1960-03-01,452.973145
3,AirPassengers,1960-04-01,452.345306
4,AirPassengers,1960-05-01,476.039246
5,AirPassengers,1960-06-01,535.992554
6,AirPassengers,1960-07-01,610.464844
7,AirPassengers,1960-08-01,624.355957
8,AirPassengers,1960-09-01,511.534424
9,AirPassengers,1960-10-01,457.592957


In [ ]:
df_train

,unique_id,ds,y
0,AirPassengers,1949-01-01,112
1,AirPassengers,1949-02-01,118
2,AirPassengers,1949-03-01,132
3,AirPassengers,1949-04-01,129
4,AirPassengers,1949-05-01,121
...,...,...,...
127,AirPassengers,1959-08-01,559
128,AirPassengers,1959-09-01,463
129,AirPassengers,1959-10-01,407
130,AirPassengers,1959-11-01,362


In [ ]:
df_

,unique_id,ds,y
0,AirPassengers,1949-01-01,112
1,AirPassengers,1949-02-01,118
2,AirPassengers,1949-03-01,132
3,AirPassengers,1949-04-01,129
4,AirPassengers,1949-05-01,121
...,...,...,...
139,AirPassengers,1960-08-01,606
140,AirPassengers,1960-09-01,508
141,AirPassengers,1960-10-01,461
142,AirPassengers,1960-11-01,390


In [ ]:
#| hide
# test pass dataframe with index
df_ds_index = df_.set_index('ds')[['unique_id', 'y']]
df_ds_index.index = pd.DatetimeIndex(df_ds_index.index)
fcst_inferred_df_index = timegpt.forecast(df_ds_index, h=10)
anom_inferred_df_index = timegpt.detect_anomalies(df_ds_index)
fcst_inferred_df = timegpt.forecast(df_[['ds', 'unique_id', 'y']], h=10)
anom_inferred_df = timegpt.detect_anomalies(df_[['ds', 'unique_id', 'y']])
pd.testing.assert_frame_equal(fcst_inferred_df_index, fcst_inferred_df, atol=1e-3)
pd.testing.assert_frame_equal(anom_inferred_df_index, anom_inferred_df, atol=1e-3)
df_ds_index = df_ds_index.groupby('unique_id').tail(80)
for freq in ['Y', 'W-MON', 'Q-DEC', 'H']:
    df_ds_index.index = np.concatenate(
        df_ds_index['unique_id'].nunique() * [pd.date_range(end='2023-01-01', periods=80, freq=freq)]
    )
    fcst_inferred_df_index = timegpt.forecast(df_ds_index, h=10)
    df_test = df_ds_index.reset_index()
    fcst_inferred_df = timegpt.forecast(df_test, h=10)
    pd.testing.assert_frame_equal(fcst_inferred_df_index, fcst_inferred_df, atol=1e-3)

In [ ]:
#| hide
# test add date features with exogenous variables 
# and multiple series
date_features = ['year', 'month']
df_date_features, future_df = _TimeGPTModel(
    client=timegpt.client,
    h=24, 
    freq='H', 
    date_features=date_features,
    date_features_to_one_hot=None,
).add_date_features(df_,  X_df=df_actual_future)
assert all(col in df_date_features for col in date_features)
assert all(col in future_df for col in date_features)
pd.testing.assert_frame_equal(
    df_date_features[df_.columns],
    df_,
)
pd.testing.assert_frame_equal(
    future_df[df_actual_future.columns],
    df_actual_future,
)

In [ ]:
#| hide
# test add date features one hot with exogenous variables 
# and multiple series
date_features = ['month', 'day']
df_date_features, future_df = _TimeGPTModel(
    client=timegpt.client,
    h=24, 
    freq='H', 
    date_features=date_features,
    date_features_to_one_hot=date_features,
).add_date_features(df_,  X_df=df_actual_future)
pd.testing.assert_frame_equal(
    df_date_features[df_.columns],
    df_,
)
pd.testing.assert_frame_equal(
    future_df[df_actual_future.columns],
    df_actual_future,
)

In [ ]:
#| hide
# test warning horizon too long
timegpt.forecast(df=df.tail(3), h=100, time_col='timestamp', target_col='value')

In [ ]:
#| hide 
# test short horizon with add_history
test_fail(
    lambda: timegpt.forecast(df=df.tail(3), h=12, time_col='timestamp', target_col='value', add_history=True),
    contains='be sure'
)

In [ ]:
#| hide 
# test short horizon with finetunning
test_fail(
    lambda: timegpt.forecast(df=df.tail(3), h=12, time_col='timestamp', target_col='value', finetune_steps=10),
    contains='be sure'
)

In [ ]:
#| hide 
# test short horizon with level
test_fail(
    lambda: timegpt.forecast(df=df.tail(3), h=12, time_col='timestamp', target_col='value', level=[80, 90]),
    contains='be sure'
)

In [ ]:
#| hide
# test custom url
# same results
_timegpt_fcst_df = _timegpt.forecast(df=df, h=12, time_col='timestamp', target_col='value')
timegpt_fcst_df = timegpt.forecast(df=df, h=12, time_col='timestamp', target_col='value')
pd.testing.assert_frame_equal(
    _timegpt_fcst_df,
    timegpt_fcst_df,
)

In [ ]:
show_doc(TimeGPT.detect_anomalies, title_level=2)